In [1]:
import torch
import pypose as pp

# 1. Intialization

In [2]:
a = pp.so3(torch.randn(2,3))
x = pp.identity_SE3(2,1)
y = pp.randn_se3(2,2)
print('a:', a, '\nx.shape:', x.shape, '\nx.gshape:', x.lshape)
print(x.lview(2))
print(y)

a: so3Type LieTensor:
tensor([[ 0.6358, -0.1589, -0.9093],
        [-0.4168, -0.1306, -1.4639]]) 
x.shape: torch.Size([2, 1, 7]) 
x.gshape: torch.Size([2, 1])
SE3Type LieTensor:
tensor([[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1.]])
se3Type LieTensor:
tensor([[[ 0.8726,  0.3301, -0.9665,  0.2766, -1.2635,  0.4250],
         [-1.4630, -0.6028,  0.3675,  0.5606,  0.2667,  1.4805]],

        [[ 0.9815, -0.3840,  0.3326,  1.7540, -1.4837,  0.8004],
         [ 0.1170, -1.9541, -0.9162,  0.6001, -1.6201,  0.1619]]])


### All arguments in PyTorch are supported

In [3]:
a = pp.randn_SO3(3, device="cuda:0", dtype=torch.double, requires_grad=True)
b = pp.identity_like(a, device="cpu")
a, b
t = a.float()
a, t

(SO3Type LieTensor:
 tensor([[-0.1433,  0.4112,  0.1737,  0.8833],
         [ 0.2252,  0.1924, -0.5417,  0.7866],
         [ 0.7282,  0.2878, -0.3736,  0.4973]], device='cuda:0',
        dtype=torch.float64, requires_grad=True),
 SO3Type LieTensor:
 tensor([[-0.1433,  0.4112,  0.1737,  0.8833],
         [ 0.2252,  0.1924, -0.5417,  0.7866],
         [ 0.7282,  0.2878, -0.3736,  0.4973]], device='cuda:0',
        grad_fn=<AliasBackward0>))

# 2. Slicing and Shaping

In [4]:
A = pp.randn_SO3(2,2)
B = pp.randn_SO3(2,1)
C = torch.cat([A,B], dim=1)         # Tensor cat
C[0,1] = pp.randn_SO3(1)            # Slicing set
D = C[1,:].Log()                    # Slicing get
E, F = torch.split(C, [1,2], dim=1) # Tensor split
print('A:', A.lshape)
print('B:', B.lshape)
print('C:', C.lshape)
print('D:', D.lshape)
print('E:', E.lshape)
print('F:', F.lshape)

A: torch.Size([2, 2])
B: torch.Size([2, 1])
C: torch.Size([2, 3])
D: torch.Size([3])
E: torch.Size([2, 1])
F: torch.Size([2, 2])


# 3. Basic Operations

In [5]:
(x * y.Exp()).Inv().Log()

se3Type LieTensor:
tensor([[[-0.8726, -0.3301,  0.9665, -0.2766,  1.2635, -0.4250],
         [ 1.4630,  0.6028, -0.3675, -0.5606, -0.2667, -1.4805]],

        [[-0.9815,  0.3840, -0.3326, -1.7540,  1.4837, -0.8004],
         [-0.1170,  1.9541,  0.9162, -0.6001,  1.6201, -0.1619]]])

# 4. Adjoint Transforms

In [6]:
X = pp.randn_Sim3(6, dtype=torch.double)
a = pp.randn_sim3(6, dtype=torch.double)
b = X.AdjT(a)
print((X * b.Exp() - a.Exp() * X).abs().mean() < 1e-7)

tensor(True)


In [7]:
X = pp.randn_SE3(8)
a = pp.randn_se3(8)
b = X.Adj(a)
print((b.Exp() * X - X * a.Exp()).abs().mean() < 1e-7)

tensor(True)


# 5. Grdients

In [8]:
x = pp.randn_so3(3, sigma=0.1, requires_grad=True, device="cuda")
assert x.is_leaf
loss = (x.Exp().Log()**2).sin().sum() # Just test, No physical meaning
loss.backward()
y = x.detach()
loss, x.grad, x, y

(tensor(0.1383, device='cuda:0', grad_fn=<SumBackward0>),
 tensor([[-0.3501, -0.0821,  0.2058],
         [ 0.0680,  0.2020, -0.3399],
         [ 0.1391, -0.2679,  0.3593]], device='cuda:0'),
 so3Type LieTensor:
 tensor([[-0.1751, -0.0410,  0.1029],
         [ 0.0340,  0.1010, -0.1700],
         [ 0.0695, -0.1340,  0.1798]], device='cuda:0', requires_grad=True),
 so3Type LieTensor:
 tensor([[-0.1751, -0.0410,  0.1029],
         [ 0.0340,  0.1010, -0.1700],
         [ 0.0695, -0.1340,  0.1798]], device='cuda:0'))

# 6. Test a Module

In [9]:
from torch import nn

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class TestNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.weight = pp.Parameter(pp.randn_so3(n))

    def forward(self, x):
        return self.weight.Exp() * x


n,epoch = 4, 5
net = TestNet(n).cuda()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.2, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,4], gamma=0.5)

print("Before Optimization:\n", net.weight)
for i in range(epoch):
    optimizer.zero_grad()
    inputs = pp.randn_SO3(n).cuda()
    outputs = net(inputs)
    loss = outputs.abs().sum()
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(loss)

print("Parameter:", count_parameters(net))
print("After Optimization:\n", net.weight)

Before Optimization:
 so3Type LieTensor:
Parameter containing:
tensor([[-1.9906,  0.1487, -0.9390],
        [ 0.2977,  1.7627,  0.6284],
        [ 0.6836,  1.1660,  0.4401],
        [-0.3382, -1.3075, -0.2497]], device='cuda:0', requires_grad=True)
tensor(7.0022, device='cuda:0', grad_fn=<SumBackward0>)
tensor(7.2108, device='cuda:0', grad_fn=<SumBackward0>)
tensor(6.7905, device='cuda:0', grad_fn=<SumBackward0>)
tensor(6.8941, device='cuda:0', grad_fn=<SumBackward0>)
tensor(6.7523, device='cuda:0', grad_fn=<SumBackward0>)
Parameter: 12
After Optimization:
 so3Type LieTensor:
Parameter containing:
tensor([[-0.7525, -0.1441, -1.3598],
        [ 0.6561,  0.0567, -0.1614],
        [ 0.4767, -0.3290, -0.7701],
        [-0.8772, -1.9518,  0.2295]], device='cuda:0', requires_grad=True)
